In [26]:
import torch as tc 
from torch import optim, nn  
from torch.utils.data import TensorDataset, DataLoader, random_split 

from torchvision import transforms as T 
from torchvision import datasets 

from torchmetrics import Accuracy 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from tqdm import tqdm

In [27]:
device = 'cuda' if tc.cuda.is_available() else 'cpu'

In [28]:
transform = T.Compose([T.ToTensor(), T.Resize(size = (512, 512))])

In [29]:
train_dataset = datasets.ImageFolder('/home/ahmadrezabaqerzadeh/CovidDataSet/train/', transform = transform)
valid_dataset = datasets.ImageFolder('/home/ahmadrezabaqerzadeh/CovidDataSet/validation/', transform = transform)

In [30]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 73
    Root location: /home/ahmadrezabaqerzadeh/CovidDataSet/train/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
           )

In [31]:
valid_dataset

Dataset ImageFolder
    Number of datapoints: 21
    Root location: /home/ahmadrezabaqerzadeh/CovidDataSet/validation/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
           )

In [32]:
train_loader = DataLoader(train_dataset, batch_size = 20, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = 21, shuffle = False)

In [33]:
x, y = next(iter(train_loader))

In [34]:
class TransformerModel(nn.Module):
    
    def __init__(self,d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, activation, num_classes):
        super(TransformerModel, self).__init__()
        
        self.encoder = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, activation, batch_first = True).encoder 
        
        self.fc = nn.LazyLinear(num_classes)
        
        
        
    def forward(self, x):
        
        x = x.mean(dim = 1)
        
        y = self.encoder(x)
        
        y = y.mean(dim = 1)
        
        y = self.fc(y)
        
        return y

In [35]:
class CNNModel(nn.Module):
    
    def __init__(self, num_filters, kernel_size, num_classes):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.LazyConv2d(out_channels = num_filters[0], kernel_size = kernel_size[0], stride = 1, padding = 1, bias = True)
        self.bn1   = nn.LazyBatchNorm2d()
        self.mpool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.LazyConv2d(out_channels = num_filters[1], kernel_size = kernel_size[1], stride = 1, padding = 1, bias = True)
        self.bn2   = nn.LazyBatchNorm2d()
        self.mpool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.LazyConv2d(out_channels = num_filters[2], kernel_size = kernel_size[2], stride = 1, padding = 1, bias = True)
        self.bn3   = nn.LazyBatchNorm2d()
        self.mpool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.LazyConv2d(out_channels = num_filters[3], kernel_size = kernel_size[3], stride = 1, padding = 1, bias = True)
        self.bn4   = nn.LazyBatchNorm2d()
        self.mpool4 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.LazyConv2d(out_channels = num_filters[4], kernel_size = kernel_size[4], stride = 1, padding = 1, bias = True)
        self.bn5   = nn.LazyBatchNorm2d()
        self.apool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.mpool1(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv2(y)
        y = self.bn2(y)
        y = self.mpool2(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv3(y)
        y = self.bn3(y)
        y = self.mpool3(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv4(y)
        y = self.bn4(y)
        y = self.mpool4(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv5(y)
        y = self.bn5(y)
        y = self.apool(y)
        y = y.relu()
        
        y = y.mean(dim = 1)
        
        y = y.flatten(1)
        
        y = self.fc(y)
        
        return y

In [36]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [37]:
def train_one_epoch(model, train_loader, loss_func, optimizer, epoch):
    
    train_loss = AverageMeter()
    train_acc = Accuracy(task = 'binary').to(device)
    
    with tqdm(train_loader, unit = 'batch') as tepoch:
        
        if epoch is not None:
            tepoch.set_description(f'Epoch={epoch}')
        
        for x, y in tepoch:
            
            x = x.to(device)
            y = y.to(device)
            
            yp = model(x)
            loss = loss_func(yp.squeeze(), y.type(tc.float))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss.update(loss.item())
            train_acc(yp.squeeze(), y)
            
            tepoch.set_postfix(loss = train_loss.avg, accuracy = train_acc.compute().item() * 100)
            
    return model, train_loss.avg, train_acc.compute().item()

In [38]:
def validation(model, valid_loaedr, loss_func):
    model.eval()

    valid_loss=AverageMeter()
    valid_acc=Accuracy(task='binary').to(device)

    for x, y in valid_loaedr:

        x=x.to(device)
        y=y.to(device)

        yp=model(x)
        loss=loss_func(yp.squeeze(), y.type(tc.float))

        valid_loss.update(loss.item())
        valid_acc(yp.squeeze(), y)

    print(f'valid loss={valid_loss.avg:.4}, accuracy={valid_acc.compute() * 100 :.4}')
    print()

    return valid_loss.avg, valid_acc.compute().item()

In [39]:
model = CNNModel([4, 8, 16, 32, 128], [3, 3, 3, 3, 3],1).to(device)

In [40]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_func = nn.BCEWithLogitsLoss()

In [41]:
best_loss = tc.inf 
best_model1 = [0]

In [42]:
N = 25
for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model1[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:04<00:00,  1.14s/batch, accuracy=21.9, loss=0.872]


valid loss=0.8062, accuracy=38.1

model saved!


Epoch=1: 100%|██████████| 4/4 [00:04<00:00,  1.10s/batch, accuracy=49.3, loss=0.863]


valid loss=0.7659, accuracy=61.9

model saved!


Epoch=2: 100%|██████████| 4/4 [00:04<00:00,  1.11s/batch, accuracy=76.7, loss=0.824]


...

valid loss=0.6693, accuracy=61.9

model saved!


Epoch=23: 100%|██████████| 4/4 [00:03<00:00,  1.26batch/s, accuracy=76.7, loss=0.618]


valid loss=0.6681, accuracy=61.9

model saved!


Epoch=24: 100%|██████████| 4/4 [00:03<00:00,  1.24batch/s, accuracy=76.7, loss=0.624]


valid loss=0.6671, accuracy=61.9

model saved!


In [43]:
model = CNNModel([8, 16, 32, 64, 128], [3, 3, 3, 3, 3],1).to(device)

In [44]:
optimizer = optim.Adam(model.parameters(), lr = 0.015)
loss_func = nn.BCEWithLogitsLoss()

In [45]:
best_loss = tc.inf 
best_model2 = [0]

In [46]:
N = 30
for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model2[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:05<00:00,  1.50s/batch, accuracy=75.3, loss=0.612]


valid loss=0.6545, accuracy=61.9

model saved!


Epoch=1: 100%|██████████| 4/4 [00:05<00:00,  1.32s/batch, accuracy=75.3, loss=0.612]


valid loss=0.6646, accuracy=61.9



Epoch=2: 100%|██████████| 4/4 [00:04<00:00,  1.13s/batch, accuracy=76.7, loss=0.578]


...

valid loss=0.7183, accuracy=61.9



Epoch=28: 100%|██████████| 4/4 [00:04<00:00,  1.14s/batch, accuracy=76.7, loss=0.535]


valid loss=0.7184, accuracy=61.9



Epoch=29: 100%|██████████| 4/4 [00:04<00:00,  1.13s/batch, accuracy=76.7, loss=0.543]


valid loss=0.7189, accuracy=61.9



In [47]:
model = TransformerModel(512, 4, 4, 0, 2048, 0.3, nn.ReLU(), 1).to(device)

In [48]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_func = nn.BCEWithLogitsLoss()

In [49]:
best_loss = tc.inf 
best_model3 = [0]

In [50]:
N = 30
for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model3[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:18<00:00,  4.57s/batch, accuracy=76.7, loss=1.13]


valid loss=3.219, accuracy=38.1

model saved!


Epoch=1: 100%|██████████| 4/4 [00:16<00:00,  4.02s/batch, accuracy=65.8, loss=1.28]


valid loss=0.9838, accuracy=61.9

model saved!


Epoch=2: 100%|██████████| 4/4 [00:18<00:00,  4.55s/batch, accuracy=76.7, loss=0.613]


...

valid loss=0.7407, accuracy=61.9



Epoch=28: 100%|██████████| 4/4 [00:22<00:00,  5.57s/batch, accuracy=76.7, loss=0.558]


valid loss=0.6874, accuracy=61.9



Epoch=29: 100%|██████████| 4/4 [00:21<00:00,  5.32s/batch, accuracy=76.7, loss=0.577]


valid loss=0.6933, accuracy=61.9

